In [1]:
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim
import torch.utils.data as data
import numpy as np
import random


In [2]:
if torch.cuda.is_available():
    DTYPE = torch.cuda.FloatTensor
else:
    DTYPE = torch.FloatTensor
    

In [3]:
text = open('texts/Lovecraft.txt','r').read()
alphabet = set(text)

ix_to_char = {k:v for k,v in enumerate(alphabet)}
char_to_ix = {k:v for v,k in enumerate(alphabet)}

In [4]:
NUM_LAYERS = 1
BATCH_SIZE = 128
HIDDEN_DIM = 256
SEQ_LEN = 65


In [5]:
def sequence_to_tensor(sequence):
    tensor = torch.zeros(len(sequence),len(alphabet)).type(DTYPE)
    for i, c in enumerate(sequence):
        tensor[i][char_to_ix[c]] = 1
    return tensor.view(BATCH_SIZE,SEQ_LEN,len(alphabet))


In [6]:
class TxtLoader(data.Dataset):
    
    def __init__(self,text):
        super(TxtLoader,self).__init__()
        self.data = text
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,index):
        return self.data[index]

In [7]:
class LSTM(nn.Module):

    def __init__(self,alphabet_size, hidden_dim, output_size):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        
        self.i2h = nn.Linear(alphabet_size,hidden_dim)
        self.lstm = nn.LSTM(hidden_dim,hidden_dim,NUM_LAYERS,batch_first=True,dropout=True)
        self.h2O = nn.Linear(hidden_dim, output_size)
        
        self.hidden = self.init_hidden()
        
        
    def init_hidden(self):
       
        return (autograd.Variable(torch.zeros(NUM_LAYERS, BATCH_SIZE, self.hidden_dim).type(DTYPE)),
                autograd.Variable(torch.zeros(NUM_LAYERS, BATCH_SIZE, self.hidden_dim).type(DTYPE)))

    def forward(self, sequence):
        out = self.i2h(sequence)
        lstm_out, self.hidden = self.lstm(out.view(BATCH_SIZE,SEQ_LEN-1,-1),self.hidden)
        out = self.h2O(lstm_out.contiguous().view(-1,self.hidden_dim))
        return out
    
    
    def gen_text(self, batch,t=None):
            
        inputs = autograd.Variable(sequence_to_tensor(batch))
        idxs = torch.zeros(inputs.data.size())
        out = self(inputs[:,:-1,:])

        if t != None:
            soft_out = F.softmax(out/t,dim=1)
            
            for i in range(soft_out.size()[0]):
                idxs[i] = np.random.choice(soft_out.size()[1],p=soft_out.data.numpy()[i])
                
        else:
            idxs = out.max(1)[1].data

        
        return out,idxs
    


In [8]:
rnn = LSTM(len(alphabet),HIDDEN_DIM,len(alphabet)).type(DTYPE)
optimizer = optim.Adam(rnn.parameters(),lr=0.01)
criterion = nn.CrossEntropyLoss()

epochs = 1000

In [9]:
def train(data_loader):
    
    best_loss = float('inf')
    
    rnn.train(True)
    for epoch in range(epochs):

        losses = np.array([])
        
        for batch in data_loader:

            rnn.zero_grad()
            rnn.hidden = rnn.init_hidden()
            
            inputs = autograd.Variable(sequence_to_tensor(batch))
            
                        
            out = rnn(inputs[:,:-1,:])
                        
            _,target = inputs[:,1:,:].topk(1)
            
            
            loss = criterion(out.view(-1,len(alphabet)),target.view(-1))
            losses = np.append(losses,loss.data[0])
            
            loss.backward()
            optimizer.step()
        
        if losses.mean() < best_loss:
            best_loss = losses.mean()
            best_wts = rnn.state_dict()
            

        print("Epoch {}/{}\nLoss: {:.2f}".format(epoch+1,epochs,losses.mean()))
        print("="*15)
        
    
    rnn.load_state_dict(best_wts)


In [10]:
dataset = TxtLoader(text[:10000])
loader = data.DataLoader(dataset,batch_size=BATCH_SIZE*SEQ_LEN,drop_last=True,num_workers=4)

train(loader)

Epoch 1/1000
Loss: 4.68
Epoch 2/1000
Loss: 4.24
Epoch 3/1000
Loss: 3.26
Epoch 4/1000
Loss: 3.20
Epoch 5/1000
Loss: 3.21
Epoch 6/1000
Loss: 3.16
Epoch 7/1000
Loss: 3.12
Epoch 8/1000
Loss: 3.11
Epoch 9/1000
Loss: 3.12
Epoch 10/1000
Loss: 3.12
Epoch 11/1000
Loss: 3.09
Epoch 12/1000
Loss: 3.07
Epoch 13/1000
Loss: 3.07
Epoch 14/1000
Loss: 3.06
Epoch 15/1000
Loss: 3.05
Epoch 16/1000
Loss: 3.04
Epoch 17/1000
Loss: 3.02
Epoch 18/1000
Loss: 3.00
Epoch 19/1000
Loss: 2.98
Epoch 20/1000
Loss: 2.96
Epoch 21/1000
Loss: 2.94
Epoch 22/1000
Loss: 2.91
Epoch 23/1000
Loss: 2.89
Epoch 24/1000
Loss: 2.86
Epoch 25/1000
Loss: 2.84
Epoch 26/1000
Loss: 2.81
Epoch 27/1000
Loss: 2.79
Epoch 28/1000
Loss: 2.76
Epoch 29/1000
Loss: 2.74
Epoch 30/1000
Loss: 2.71
Epoch 31/1000
Loss: 2.69
Epoch 32/1000
Loss: 2.67
Epoch 33/1000
Loss: 2.65
Epoch 34/1000
Loss: 2.64
Epoch 35/1000
Loss: 2.62
Epoch 36/1000
Loss: 2.61
Epoch 37/1000
Loss: 2.59
Epoch 38/1000
Loss: 2.58
Epoch 39/1000
Loss: 2.56
Epoch 40/1000
Loss: 2.55
Epoch 41/

Epoch 199/1000
Loss: 0.83
Epoch 200/1000
Loss: 0.83
Epoch 201/1000
Loss: 0.81
Epoch 202/1000
Loss: 0.80
Epoch 203/1000
Loss: 0.79
Epoch 204/1000
Loss: 0.77
Epoch 205/1000
Loss: 0.76
Epoch 206/1000
Loss: 0.75
Epoch 207/1000
Loss: 0.74
Epoch 208/1000
Loss: 0.73
Epoch 209/1000
Loss: 0.72
Epoch 210/1000
Loss: 0.70
Epoch 211/1000
Loss: 0.70
Epoch 212/1000
Loss: 0.69
Epoch 213/1000
Loss: 0.69
Epoch 214/1000
Loss: 0.72
Epoch 215/1000
Loss: 0.70
Epoch 216/1000
Loss: 0.68
Epoch 217/1000
Loss: 0.65
Epoch 218/1000
Loss: 0.67
Epoch 219/1000
Loss: 0.64
Epoch 220/1000
Loss: 0.63
Epoch 221/1000
Loss: 0.62
Epoch 222/1000
Loss: 0.60
Epoch 223/1000
Loss: 0.60
Epoch 224/1000
Loss: 0.58
Epoch 225/1000
Loss: 0.57
Epoch 226/1000
Loss: 0.57
Epoch 227/1000
Loss: 0.55
Epoch 228/1000
Loss: 0.55
Epoch 229/1000
Loss: 0.54
Epoch 230/1000
Loss: 0.52
Epoch 231/1000
Loss: 0.52
Epoch 232/1000
Loss: 0.51
Epoch 233/1000
Loss: 0.50
Epoch 234/1000
Loss: 0.50
Epoch 235/1000
Loss: 0.49
Epoch 236/1000
Loss: 0.48
Epoch 237/10

Epoch 395/1000
Loss: 0.20
Epoch 396/1000
Loss: 0.19
Epoch 397/1000
Loss: 0.17
Epoch 398/1000
Loss: 0.16
Epoch 399/1000
Loss: 0.16
Epoch 400/1000
Loss: 0.15
Epoch 401/1000
Loss: 0.14
Epoch 402/1000
Loss: 0.13
Epoch 403/1000
Loss: 0.13
Epoch 404/1000
Loss: 0.12
Epoch 405/1000
Loss: 0.12
Epoch 406/1000
Loss: 0.11
Epoch 407/1000
Loss: 0.11
Epoch 408/1000
Loss: 0.11
Epoch 409/1000
Loss: 0.10
Epoch 410/1000
Loss: 0.10
Epoch 411/1000
Loss: 0.10
Epoch 412/1000
Loss: 0.09
Epoch 413/1000
Loss: 0.09
Epoch 414/1000
Loss: 0.09
Epoch 415/1000
Loss: 0.09
Epoch 416/1000
Loss: 0.08
Epoch 417/1000
Loss: 0.08
Epoch 418/1000
Loss: 0.08
Epoch 419/1000
Loss: 0.08
Epoch 420/1000
Loss: 0.08
Epoch 421/1000
Loss: 0.08
Epoch 422/1000
Loss: 0.08
Epoch 423/1000
Loss: 0.07
Epoch 424/1000
Loss: 0.07
Epoch 425/1000
Loss: 0.07
Epoch 426/1000
Loss: 0.07
Epoch 427/1000
Loss: 0.07
Epoch 428/1000
Loss: 0.07
Epoch 429/1000
Loss: 0.07
Epoch 430/1000
Loss: 0.07
Epoch 431/1000
Loss: 0.07
Epoch 432/1000
Loss: 0.07
Epoch 433/10

Epoch 591/1000
Loss: 0.04
Epoch 592/1000
Loss: 0.04
Epoch 593/1000
Loss: 0.04
Epoch 594/1000
Loss: 0.04
Epoch 595/1000
Loss: 0.04
Epoch 596/1000
Loss: 0.04
Epoch 597/1000
Loss: 0.04
Epoch 598/1000
Loss: 0.04
Epoch 599/1000
Loss: 0.04
Epoch 600/1000
Loss: 0.04
Epoch 601/1000
Loss: 0.04
Epoch 602/1000
Loss: 0.04
Epoch 603/1000
Loss: 0.04
Epoch 604/1000
Loss: 0.04
Epoch 605/1000
Loss: 0.04
Epoch 606/1000
Loss: 0.04
Epoch 607/1000
Loss: 0.04
Epoch 608/1000
Loss: 0.04
Epoch 609/1000
Loss: 0.04
Epoch 610/1000
Loss: 0.04
Epoch 611/1000
Loss: 0.04
Epoch 612/1000
Loss: 0.04
Epoch 613/1000
Loss: 0.04
Epoch 614/1000
Loss: 0.04
Epoch 615/1000
Loss: 0.04
Epoch 616/1000
Loss: 0.04
Epoch 617/1000
Loss: 0.04
Epoch 618/1000
Loss: 0.04
Epoch 619/1000
Loss: 0.04
Epoch 620/1000
Loss: 0.04
Epoch 621/1000
Loss: 0.04
Epoch 622/1000
Loss: 0.04
Epoch 623/1000
Loss: 0.04
Epoch 624/1000
Loss: 0.04
Epoch 625/1000
Loss: 0.04
Epoch 626/1000
Loss: 0.04
Epoch 627/1000
Loss: 0.04
Epoch 628/1000
Loss: 0.04
Epoch 629/10

Epoch 787/1000
Loss: 0.04
Epoch 788/1000
Loss: 0.04
Epoch 789/1000
Loss: 0.04
Epoch 790/1000
Loss: 0.04
Epoch 791/1000
Loss: 0.04
Epoch 792/1000
Loss: 0.04
Epoch 793/1000
Loss: 0.04
Epoch 794/1000
Loss: 0.04
Epoch 795/1000
Loss: 0.04
Epoch 796/1000
Loss: 0.04
Epoch 797/1000
Loss: 0.04
Epoch 798/1000
Loss: 0.04
Epoch 799/1000
Loss: 0.04
Epoch 800/1000
Loss: 0.04
Epoch 801/1000
Loss: 0.04
Epoch 802/1000
Loss: 0.04
Epoch 803/1000
Loss: 0.04
Epoch 804/1000
Loss: 0.04
Epoch 805/1000
Loss: 0.04
Epoch 806/1000
Loss: 0.04
Epoch 807/1000
Loss: 0.04
Epoch 808/1000
Loss: 0.04
Epoch 809/1000
Loss: 0.04
Epoch 810/1000
Loss: 0.04
Epoch 811/1000
Loss: 0.04
Epoch 812/1000
Loss: 0.04
Epoch 813/1000
Loss: 0.04
Epoch 814/1000
Loss: 0.04
Epoch 815/1000
Loss: 0.04
Epoch 816/1000
Loss: 0.04
Epoch 817/1000
Loss: 0.04
Epoch 818/1000
Loss: 0.04
Epoch 819/1000
Loss: 0.04
Epoch 820/1000
Loss: 0.04
Epoch 821/1000
Loss: 0.04
Epoch 822/1000
Loss: 0.04
Epoch 823/1000
Loss: 0.04
Epoch 824/1000
Loss: 0.04
Epoch 825/10

Epoch 983/1000
Loss: 0.03
Epoch 984/1000
Loss: 0.03
Epoch 985/1000
Loss: 0.03
Epoch 986/1000
Loss: 0.03
Epoch 987/1000
Loss: 0.03
Epoch 988/1000
Loss: 0.03
Epoch 989/1000
Loss: 0.03
Epoch 990/1000
Loss: 0.03
Epoch 991/1000
Loss: 0.03
Epoch 992/1000
Loss: 0.03
Epoch 993/1000
Loss: 0.03
Epoch 994/1000
Loss: 0.03
Epoch 995/1000
Loss: 0.03
Epoch 996/1000
Loss: 0.03
Epoch 997/1000
Loss: 0.03
Epoch 998/1000
Loss: 0.03
Epoch 999/1000
Loss: 0.03
Epoch 1000/1000
Loss: 0.03


In [11]:
string = text[0]  


rnn.train(False)

for batch in loader:
    
    _ ,idxs = rnn.gen_text(batch)
    
#for i in range(100):
    
    #out = rnn(out)
    #soft_out = F.softmax(out/t,dim=1)

for c in idxs:
    string += ix_to_char[c]

print(string)           



#print(string,file=open('texts/output.txt','w'))

T,e horror In Clay

The most merciful thing in the world, I think bn nhenonability of the human mind to correlate all its content  Thboive,of e placid island of ignorance in the midst of black  ndtof infinity, and it was not meant that we should voyage far.ohecthutnces, each straining in its own direction, have hithertoiob  d kn nittle; but some day the piecing together of dissociat  "nownedge will open up such terrifying vistas of reality, and ueoft iiightful.position therein, that we shall either go mad fru ohaoseaenation or flee from the light into the peace and safet an cnfaa Oark age.

Theosophists have guessed at the awesome grdn; s of themwiumic cycle wherein our world and human race form  iz  ens sncidents. They have hinted at strange survivals in tero hhich sauld freeze the blood if not masked by a bland optimism Neiuwn ws not fromethe  ahet the e came the single glimpse of fntodden cons which chills me when I think of it and maddens me wer o dream of it. That glimpse, like al

In [12]:
# pre-process txt file a bit
# hyperparameters
# refactor